In [4]:
import sys
sys.path.append('../')
from run import Prepro
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import  GridSearchCV
from utils import utils_tensor as utils #import local functions

In [5]:
# load vision and stat data 
# Leonard: perhaps we can save X_train + X_test in one .npy file, and do train/test split here? Similar with y. 
# we can keep features such as SID, year, basin etc in the front. That way we have more flexibility? 
# We can even make a df, using pd.DataFrame(values, columns = keys). Values are based on the pre-existing function, and keys we can output it (as a list) in processing function?  

wind_size=8
reduced_ranks = [5,5,3,3]


#load x_viz: pre-processed compressed vision data 
x_viz_train = np.load('data/X_train_viz_reduce_1980_50_20_90_w'+str(window_size)+'_'+utils.ranks_to_str(reduced_ranks)+'.npy', allow_pickle=True)
x_viz_test = np.load('data/X_test_viz_reduce_1980_50_20_90_w'+str(window_size)+'_'+utils.ranks_to_str(reduced_ranks)+'.npy', allow_pickle=True)
#load x_stat: stat_data 
x_stat_train = np.load('data/X_train_stat_1980_50_20_90_w' + str(window_size) + '.npy', allow_pickle=True).reshape(-1, window_size, 22)[:,:,:7]
x_stat_test = np.load('data/X_test_stat_1980_50_20_90_w' + str(window_size) + '.npy', allow_pickle=True).reshape(-1, window_size, 22)[:,:,:7]
#standardize x: 
x_viz_train, x_viz_test = utils.standardize_x_viz(x_viz_train, x_viz_test)
x_stat_train, x_stat_test = utils.standardize_x_stat(x_stat_train, x_stat_test)
#concat viz and stat 
X_train=utils.concat_stat_viz(x_stat_train, x_viz_train)
X_test=utils.concat_stat_viz(x_stat_test, x_viz_test)

#load y: intensity  
tgt_intensity_train = np.load('data/y_train_intensity_1980_50_20_90_w' + str(window_size) + '.npy', allow_pickle=True)
tgt_intensity_test = np.load('data/y_test_intensity_1980_50_20_90_w' + str(window_size) + '.npy', allow_pickle=True)
#standardize y 
tgt_intensity_train,tgt_intensity_test, mean_intensity, std_intensity  = utils.standardize_y(tgt_intensity_train,tgt_intensity_test)

#load y: displacement 
# tgt_displacement_train = np.load('data/y_train_displacement_1980_50_20_90_w' + str(window_size) + '.npy', allow_pickle=True)
# tgt_displacement_test = np.load('data/y_test_displacement_1980_50_20_90_w' + str(window_size) + '.npy', allow_pickle=True)
#standardize y 
# tgt_dx_train,tgt_dx_test, mean_dx, std_dx  = utils.standardize_y(tgt_displacement_train[:,0],tgt_displacement_test[:,0])
# tgt_dy_train,tgt_dy_test, mean_dy, std_dy  = utils.standardize_y(tgt_displacement_train[:,1],tgt_displacement_test[:,1])

# load y: category  
# tgt_intensity_cat_train = np.load('data/y_train_intensity_cat_1980_50_20_90_w' + str(window_size) + '.npy', allow_pickle=True)
# tgt_intensity_cat_test = np.load('data/y_test_intensity_cat_1980_50_20_90_w' + str(window_size) + '.npy', allow_pickle=True)
# tgt_intensity_cat_baseline_train = np.load('data/y_train_intensity_cat_baseline_1980_50_20_90_w' + str(window_size) + '.npy',  allow_pickle = True)
# tgt_intensity_cat_baseline_test = np.load('data/y_test_intensity_cat_baseline_1980_50_20_90_w' + str(window_size) + '.npy', allow_pickle=True)


NameError: name 'window_size' is not defined

In [7]:
##feature selection: skit learn has built-in feature selection function, we may not need to use iai. 
# https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/ 
# here is a small example: 
from sklearn.feature_selection import SelectFromModel

X_train_sparse = X_train.copy()

# fit model on training data
model = XGBRegressor(max_depth=5, n_estimators=100)
model.fit(X_train, tgt_intensity_train)

# select features using threshold
select = SelectFromModel(model, threshold="mean", prefit=True)
X_train_sparse = select.transform(X_train)

In [ ]:
##grid search 
grid = GridSearchCV(estimator = XGBRegressor(learning_rate = 0.15, n_estimators=140, max_depth=5,
 min_child_weight=1, subsample=0.8, seed=1),
 param_grid = params, n_jobs=4, scoring = 'neg_mean_absolute_error')

grid.fit(X_train_sparse, tgt_intensity_train)

params = {
 'min_child_weight':range(1,6,2),
 'n_estimators':[100, 140],
 'subsample':[0.6,0.8,1],
 'learning_rate':[0.1, 0.15, 0.2]
}

grid.grid_scores_, grid.best_params_, grid.best_score_

In [1]:
# old code 
# trying Julia below

# import numpy as np
# import pandas as pd
# from xgboost import XGBClassifier
# from xgboost import XGBRegressor
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import mean_absolute_error
# from sklearn.model_selection ßimport  GridSearchCV

# from julia import Julia
# Julia(sysimage='../sys.so', compiled_modules = False)
# from interpretableai import iai